#Creación de una Base de Datos Analítica
##Evidencia de Aprendizaje 1

## Integrantes:
### Jorge Armando Rodriguez
### Lina Johana Seguro Gaviria

##Problemática Abordada

La empresa SportZone S.A., dedicada a la venta de artículos de fútbol americano como balones, uniformes, cascos y accesorios, tanto en tiendas físicas como en plataformas digitales, presenta dificultades en el procesamiento y manejo de datos debido a la falta de una base de datos centralizada que integre toda la información importante.

Para solucionar esta situación, se implementará una base de datos bien estructurada que le permita a la empresa centralizar y migrar su información de productos, clientes, proveedores y ventas. Esta solución le brindará la capacidad de organizar e integrar los datos provenientes de diversas fuentes, garantizando su integridad y mejorando la eficiencia en su gestión, con el propósito de facilitar futuras consultas y procesos operativos.

## Objetivo:
Implementar una base de datos que centralice y organice la información de SportZone S.A., mejorando la integridad y eficiencia en el manejo de los datos.

##Dataset Escogido

**Nombre:** Datos de venta minorista: Ventas de equipamiento de fútbol americano                                        
**Fuente:** Kaggle-Retail Data: American Football Gear Sales           
**Enlace:** https://www.kaggle.com/datasets/larysa21/retail-data-american-football-gear-sales?select=AF_offline_sales_dataset.csv                             
**Autor:**No especificado (dataset sintético generado con Mockaroo).                                                   

Escogimos este dataset porque entendemos el modelo de ventas y las relaciones entre productos, clientes y operaciones comerciales, lo que nos facilita su uso en la migración y carga de datos para construir una base de datos coherente y funcional.


##Variables Relevantes

El dataset Retail Data: American Football Gear Sales (Datos Minoristas: Ventas de Equipamiento de Fútbol Americano)incluye información clave sobre productos, clientes, proveedores y ventas.Las variables más relevantes y su utilidad para el caso de uso son: 

![Variables Relevantes](VR.png)


#Modelo Entidad-Relación (ERD)


![ERD](ERD.png)

Este Modelo entidad_relación representa un modelo estrella para el análisis de ventas de artículos deportivos.  
La tabla central `PRODUCTO_CLIENTE_VENTAS` contiene todos los atributos de ventas, productos y clientes, y sirve como fact table.  
Las tablas `PRODUCTO`, `CLIENTE` y `VENTAS` actúan como dimensiones, relacionadas únicamente con la tabla central, permitiendo consultas y reportes eficientes.


#Creación de Base de Datos + Carga de Datos

In [0]:
!pip install kagglehub[pandas-datasets]>=0.3.8

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


##Importaciones de Librerias 

In [0]:
import os
import zipfile
import kagglehub
import pandas as pd 



## Funciones para Descargar, Extraer y el Leer el Dataset desde Kaggle.

In [0]:
def download_dataset_zip(url = ""):
        print("Descargando dataset desde Kaggle...")
        dataset_path = kagglehub.dataset_download(url)
        print("Ruta al dataset:", dataset_path)
        return dataset_path
    
def extract_zip_files(dataset_path):
        zip_files = [f for f in os.listdir(dataset_path) if f.endswith('.zip')]
        if zip_files:
            zip_file = os.path.join(dataset_path, zip_files[0])
            extract_dir = os.path.join(dataset_path, "extracted")
            os.makedirs(extract_dir, exist_ok=True)
            print(f"Extrayendo {zip_file} en {extract_dir}...")
            with zipfile.ZipFile(zip_file, "r") as z:
                z.extractall(extract_dir)
            return extract_dir
        else:
            # Si no se encuentra un ZIP, se verifica si existen archivos CSV en la ruta
            csv_files = [f for f in os.listdir(dataset_path) if f.endswith('.csv')]
            if csv_files:
                print("No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.")
                return dataset_path
            else:
                raise FileNotFoundError("No se encontró ningún archivo .zip ni archivos .csv en la ruta del dataset")

def create_csv(csv_dir, csv_name=None):
    if csv_name:
        file_path = os.path.join(csv_dir, csv_name)
        print(f"Leyendo {file_path}...")
        df = pd.read_csv(file_path, encoding="latin1")
        print("CSV creado correctamente")
        return df
    else:
        csv_files = [f for f in os.listdir(csv_dir) if f.endswith('.csv')]
        if not csv_files:
            raise FileNotFoundError("No se encontraron archivos CSV en el directorio extraído")
        for file in csv_files:
            file_path = os.path.join(csv_dir, file)
            print(f"Leyendo {file_path}...")
            df = pd.read_csv(file_path, encoding="latin1")
        print("CSV creado correctamente")
        return df

    

## Descarga del Dataset

In [0]:
df = pd.DataFrame()
dataset_path = download_dataset_zip("larysa21/retail-data-american-football-gear-sales") 
csv_dir = extract_zip_files(dataset_path)
df = create_csv(csv_dir, csv_name="AF_offline_sales_dataset.csv")


Descargando dataset desde Kaggle...


100%|██████████| 97.3M/97.3M [00:01<00:00, 57.9MB/s]

Extracting files...


Ruta al dataset: /home/spark-5b7c0640-b1f0-4b1b-b39e-45/.cache/kagglehub/datasets/larysa21/retail-data-american-football-gear-sales/versions/1
No se encontró archivo ZIP pero se detectaron archivos CSV; se asume que el dataset ya se encuentra extraído.
Leyendo /home/spark-5b7c0640-b1f0-4b1b-b39e-45/.cache/kagglehub/datasets/larysa21/retail-data-american-football-gear-sales/versions/1/AF_offline_sales_dataset.csv...
CSV creado correctamente


## Verificación de Carga 

In [0]:
df.head(4)

,product_name,brand,category,subcategory,supplier,date,price,quantity_sold,amount_sold,cost_amount,payment_method,customer_firstname,customer_lastname,customer_gender,customer_email,customer_phone,store_type,store_street,store_city,store_state
0,Riddell Victor-I Inflation Air Bladder (R91229...,Riddell,Helmets,Helmet Components,Balistreri Inc.,2023-07-31 16:54:01,16.11,4,64.44,29.76,cash,Farleigh,Geach,Male,fgeach55@aol.com,296-345-9732,specialized,20 Fairfield Plaza,Portland,OR
1,McDavid Thigh Support,McDavid,Protective Gear,Thigh,NFL Properties LLC,2023-03-16 20:06:34,20.00,6,120.00,81.36,debit card,Stesha,Peiser,Female,speiserl@squarespace.com,562-102-4205,superstore,654 Pine View Place,New Orleans,LA
2,SportStar SpeedFlex Adapter/Set,SportStar,Helmets,Helmet Components,Forelle Inc.,2023-04-01 08:44:54,55.18,9,496.62,122.31,cash,Luciana,Colleford,Female,lcollefordm@zdnet.com,726-840-1316,superstore,7 Transport Court,Clearwater,FL
3,Riddell G3BD (R74975),Riddell,Helmets,Face Masks,Franklin Sports,2022-04-13 01:48:43,39.34,1,39.34,20.05,bank transfer,Percy,Folbigg,Male,pfolbigg9u@live.com,437-794-0956,superstore,6451 Gina Center,Amarillo,TX


## Convertir Dataframe de pandas a Spark

In [0]:
spark_df = spark.createDataFrame(df)

##Creación la Tabla e Inserto de Datos

In [0]:
spark_df.write.mode("overwrite").saveAsTable("tbl_ventas_futbol")

## Verifica la creación correcta de la tabla 

In [0]:
%sql
SELECT * 
FROM tbl_ventas_futbol
LIMIT 5;

product_name,brand,category,subcategory,supplier,date,price,quantity_sold,amount_sold,cost_amount,payment_method,customer_firstname,customer_lastname,customer_gender,customer_email,customer_phone,store_type,store_street,store_city,store_state
Shock Doctor NoSweat Helmet Liner Black - 6 pack,Shock Doctor,Helmets,Helmet Components,EP Sports,2023-09-07 17:51:56,47.47,2,94.94,12.06,mobile payment,Esme,Collabine,Male,ecollabine3h@so-net.ne.jp,805-365-7445,franchise,0 Gale Parkway,Albuquerque,NM
Nike Hyperstrong Core Padded Forearm Shivers,Nike,Protective Gear,Forearm,Grady and Sons,2022-01-22 05:04:01,19.71,6,118.26,93.72,credit card,Maryann,Halbard,Female,mhalbard3i@google.cn,538-788-1419,specialized,10 Hansons Hill,San Bernardino,CA
All Star KP2500 Small Adult Knee Pad (Pairs),All Star,Protection,Padding,Krausko LLC,2022-03-02 21:14:41,12.68,1,12.68,7.23,cash,Edmon,Ingles,Male,eingles3j@time.com,859-587-9700,superstore,7522 Algoma Park,Erie,PA
Douglas Eclipse PEC01 Black Edition,Douglas,Shoulder Pads,Adult,Nader Group,2023-02-18 03:22:21,280.39,3,841.17,107.85,mobile payment,Levy,Morillas,Male,lmorillas3k@amazon.co.uk,917-442-8216,franchise,16 Graedel Plaza,Phoenix,AZ
Rawlings YMRC Youth Rib Protector,Rawlings,Shoulder Pads,Shoulder Pad Components,VOIT Corp.,2022-09-02 13:19:03,21.02,10,210.2,79.8,cash,Celene,Corrie,Female,ccorrie3l@simplemachines.org,353-195-4046,superstore,8739 Linden Alley,Arlington,TX


##Consultas SQL 

## 1. Conteo de Registros

In [0]:
%sql
SELECT COUNT(*) FROM tbl_ventas_futbol;

COUNT(*)
501000


El número 501000 arrojado en la consulta, indica que la tabla tbl_ventas_futbol tiene 501000 registros en total, lo que indica que hay esa misma cantidad de transacciones realizadas, ya que cada fila representa una venta individual.


## 2. Nombres y Tipos de Columnas 

In [0]:
%sql
DESCRIBE TABLE tbl_ventas_futbol;


col_name,data_type,comment
product_name,string,null
brand,string,null
category,string,null
subcategory,string,null
supplier,string,null
date,string,null
price,double,null
quantity_sold,bigint,null
amount_sold,double,null
cost_amount,double,null


Esta consulta nos devuelve la  descripción de la estructura de la tabla tbl_ventas_futbol, que incluye el nombre de las columnas y el tipo de dato de cada una. Además, la consulta muestra información adicional, como si las columnas permiten valores nulos o no.
Esta consulta es importante por que nos permite validar que la tabla se haya creado correctamente a partir de DataFrame original y que cada campo tenga la información que esperabamos. 

## 3. Consulta con Filtro

In [0]:
%sql
SELECT *
FROM tbl_ventas_futbol
WHERE store_city = 'Miami'
LIMIT 10;

product_name,brand,category,subcategory,supplier,date,price,quantity_sold,amount_sold,cost_amount,payment_method,customer_firstname,customer_lastname,customer_gender,customer_email,customer_phone,store_type,store_street,store_city,store_state
Shock Doctor Ultra Pro Showtime Chin Strap,Shock Doctor,Helmets,Chin Straps,Nerion Athletics Group,2023-07-27 12:30:33,14.48,8,115.84,58.24,cash,Kanya,McKie,Female,kmckieku@theglobeandmail.com,737-955-6998,megastore,7 Gina Parkway,Miami,FL
Reebok NFL Burner Speed Low SD3,Reebok,Footwear,Detachable Cleats,Grady and Sons,2023-07-12 13:03:14,233.09,8,1864.72,467.2,bank transfer,Rhonda,Soles,Female,rsolesme@jalbum.net,971-544-5232,megastore,7 Gina Parkway,Miami,FL
Rawlings IMPULSE Helmets Odd. Colors (S-M-L),Rawlings,Helmets,Adult,Howell LLC,2023-06-04 13:54:01,720.74,7,5045.18,1776.32,credit card,Horatio,Ellissen,Male,hellissenn3@dagondesign.com,735-509-6197,megastore,7 Gina Parkway,Miami,FL
Riddell Speedflex Threaded Valve Retainer Cap (R920401),Riddell,Helmets,Helmet Maintenance,BRG Sports Inc.,2023-10-24 18:22:26,9.2,4,36.8,8.64,mobile payment,Wilhelm,Frankel,Male,wfrankeln8@addtoany.com,182-591-9819,megastore,7 Gina Parkway,Miami,FL
Cutters Playmaker Triple Youth Wristcoach,Cutters,Accessories,Wristbands,Halvorson Group,2022-08-14 22:00:44,9.5,7,66.5,39.83,debit card,Erny,Armatidge,Male,earmatidge65@surveymonkey.com,816-778-5994,megastore,7 Gina Parkway,Miami,FL
Xenith Flyte 2 Youth TD,Xenith,Shoulder Pads,Youth,Walter-Sipes,2022-09-16 05:43:11,56.64,9,509.76,264.24,credit card,Gonzalo,Shorter,Male,gshorter6c@ning.com,164-817-4120,megastore,7 Gina Parkway,Miami,FL
Nike Force Savage Pro 2 (AH4000),Nike,Footwear,Molded Cleats,Mohr-Rutherford,2023-02-24 15:01:21,114.44,2,228.88,237.24,credit card,Artur,Chalmers,Genderfluid,achalmers8x@goodreads.com,644-655-5503,megastore,7 Gina Parkway,Miami,FL
Riddell Power 5 Piece Int. Girdle Youth (RGWPTY),Riddell,Protection,Girdle Pads,EP Sports,2023-09-14 10:50:56,73.79,5,368.95,122.3,mobile payment,Elysee,Livzey,Female,elivzey9v@independent.co.uk,796-248-1570,megastore,7 Gina Parkway,Miami,FL
Nike Eye Shield w/Multicolor Decal Pack - Clear,Nike,Helmets,Helmet Components,Grady and Sons,2023-06-17 11:07:17,46.6,4,186.4,61.56,mobile payment,Ruy,Kloska,Agender,rkloskaad@dailymotion.com,753-131-3136,megastore,7 Gina Parkway,Miami,FL
Rawlings FFAPY Youth FTBL Forearm Pad,Rawlings,Protective Gear,Forearm,Hintz Group,2023-03-15 12:28:24,40.25,10,402.5,64.9,debit card,Ettore,Robertis,Male,erobertisax@yahoo.co.jp,897-543-2234,megastore,7 Gina Parkway,Miami,FL


Esta consulta nos permite visualizar los registros correspondientes a las ventas efectuadas en la ciudad de Miami.
SQL filtra la información de la tabla ventas_futbol para mostrar únicamente las filas en las que el campo store_city presenta el valor 'Miami'.Lo cual nos permite hacer un análisis de las transacciones realizadas en esa ciudad, mostrando los campos disponibles para cada venta realizada en esa ciudad. En total se arrojaron 4995 registros, en la tabla se muestran los primeros 10 como muestra representativas con el fin de no cargar la vista de Databricks. 